In [1]:
import numpy as np
import inspect
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

from math import exp, sqrt
from sympy import *

# Лабораторная №2

## Utils

### Полезные ссылки

- [метод ньютона](http://fourier.eng.hmc.edu/e176/lectures/ch3/node6.html)

### Finding derivative

In [17]:
def get_dfs(func):
    arg_symbols = symbols(inspect.getargspec(func).args)
    sym_func = func(*arg_symbols)

    return [lambdify(arg_symbols, sym_func.diff(a)) for a in arg_symbols]

In [18]:
def calc_df(func, params):
    dfs = get_dfs(func)
    res = []
    for df in dfs:
        res.append(df(*params))
    return res

In [26]:
def calc_inv_hessian(func, params):
    dfs = get_dfs(func)
    n = len(dfs)
    
    h = [[0 for x in range(n)] for y in range(n)] 
    for i in range(len(dfs)):
        h[i] = get_dfs(dfs[i])
        
    inv_h = np.linalg.inv(h)
    
    for i in range(n):
        for j in range(n):
            inv_h[i][j] = inv_h[i][j](*params)
    
    return inv_h 

### Visualization methods

In [2]:
def print_res(res, step, xs):
    print('Result:', res)
    print('Total iterations: ', step)
    print('Steps: ', xs)

In [3]:
def draw_countors_plot_with_steps(coords, f, classic = True):
    x_min = -5
    y_min = -5
    x_max = 5
    y_max = 5
    delta = 20
    
    
    x = np.linspace(x_min, x_max, delta)
    y = np.linspace(y_min, y_max, delta)
    X, Y = np.meshgrid(x, y)
    
    if (classic):
        z = f
    else:
        z = get_function_from_symmetric_matrix(f) 
        
    Z = z(X, Y)

    contours = plt.contour(X, Y, Z, 3, colors='black')
    plt.clabel(contours, inline=True, fontsize=12)
    plt.imshow(Z, extent=[x_min, x_max, y_min, y_max], origin='lower',
               cmap='RdGy', alpha=0.5)
    plt.colorbar();

    cx, cy = zip(*coords)
    plt.plot(cx, cy, color='black', marker='o')
    #plt.savefig('images/gradient_steps.png')
    plt.show()

## Метод сопряженных направлений

In [27]:
def conjugate_vecs_method(f, x, eps=1e-5):
    step = 0
    xs = [x]
    
    
    return x, step, xs

## Метод Ньютона

- функция должна быть дважды дифференцируемой

In [28]:
def newtons_method(f, x, eps=1e-5):
    step = 0
    xs = [x]
    lr = 0.5
    coef_crush = 0.5
    while (np.linalg.norm(calc_df(f,x)) > eps):
        x -= lr*(coef_crush**step)*calc_inv_hessian(f,x)*calc_df(f,x)
        step += 1
        xs.append(list(x))
    
    return x, step, xs

## Старый добрый градиентный спуск

In [29]:
def classic_gradient_descent(f, x, eps=1e-5):
    step = 0
    lr = 0.5
    coef_crush = 0.5
    xs = [x]
    
    while (np.linalg.norm(calc_df(f,x)) > eps):
        x -= lr*(coef_crush**step)*calc_df(f,x)
        step += 1
        xs.append(list(x))
        
    return x, step, xs

In [16]:
def quadratic_gradient_descent(Q, x, eps=1e-5):
    step = 0
    lr = 0.5
    coef_crush =0.5
    xs = [x]
    
    while (np.linalg.norm(Q * x) > eps):
        x -= np.linalg.norm(np.eye(len(Q)) - lr*(coef_crush**step)*Q)*np.array(x)
        step += 1
        xs.append(list(x))
        
    return x, step, xs

## Testing

### Test data

In [19]:
f = [
    lambda x, y: 100*(y-x)**2 + (1-x)**2,
    lambda x, y: 100*(y-x**2)**2 + (1-x)**2,
]

f_max = lambda x, y: 2*exp**(-((x-1)/2)**2 - (y-1)**2) + 3*exp**(-((x-2)/3)**2 - (((y-3)/2)**2))

### Graphics

#### Statistics

#### Trajectory